# Calculadora de Tiempo de Lectura para Bibliografía de Examen

Esta notebook te ayuda a calcular cuánto tiempo necesitas para leer toda la bibliografía de tu examen.

## Bibliografía incluida:

1. **Marco General** - DC Nivel Primario (completo)
2. **Diseño Curricular Primer Ciclo** - páginas 258-279
3. **Diseño Curricular Segundo Ciclo** - páginas 438-470
4. **Orientación Enseñanza y Evaluación** - 1° ciclo (pág. 84-88) y 2° ciclo (pág. 104-108)
5. **Régimen Académico** - documento completo
6. **Diseño Curricular NES** - secciones relevantes
7. **Documentos Educación Tecnológica Secundaria**
8. **Documentos Tecnologías de la Información NES**
9. **Estatuto Docente** - Capítulo III, Artículos 6 y 7
10. **Reglamento Escolar** - artículos específicos mencionados


In [ ]:
# Importar las librerías necesarias
import math
import pandas as pd
from dataclasses import dataclass
from typing import List, Dict
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo para gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
@dataclass
class Document:
    """Representa un documento con su información de lectura"""
    name: str
    pages: int
    words_per_page: int = 250
    difficulty_multiplier: float = 1.0
    
    def get_total_words(self) -> int:
        """Calcula el total de palabras en el documento"""
        return self.pages * self.words_per_page

class ReadingTimeCalculator:
    """Calculadora de tiempo de lectura para documentos académicos"""
    
    def __init__(self, words_per_minute: int = 180):
        self.words_per_minute = words_per_minute
        self.documents = []
    
    def add_document(self, document: Document):
        """Agrega un documento a la lista de lectura"""
        self.documents.append(document)
    
    def calculate_reading_time(self, document: Document) -> Dict[str, float]:
        """Calcula el tiempo de lectura para un documento específico"""
        total_words = document.get_total_words()
        adjusted_words = total_words * document.difficulty_multiplier
        minutes = adjusted_words / self.words_per_minute
        hours = minutes / 60
        
        return {
            'minutes': minutes,
            'hours': hours,
            'words': total_words,
            'adjusted_words': adjusted_words,
            'pages': document.pages
        }
    
    def get_results_dataframe(self) -> pd.DataFrame:
        """Devuelve los resultados en formato DataFrame para análisis"""
        results = []
        for doc in self.documents:
            time_info = self.calculate_reading_time(doc)
            results.append({
                'Documento': doc.name,
                'Páginas': doc.pages,
                'Palabras': time_info['words'],
                'Dificultad': doc.difficulty_multiplier,
                'Minutos': time_info['minutes'],
                'Horas': time_info['hours']
            })
        return pd.DataFrame(results)
    
    def calculate_total_time(self) -> Dict[str, any]:
        """Calcula el tiempo total de lectura"""
        df = self.get_results_dataframe()
        
        total_minutes = df['Minutos'].sum()
        total_hours = total_minutes / 60
        total_days_8h = total_hours / 8
        total_days_4h = total_hours / 4
        total_days_2h = total_hours / 2
        
        return {
            'total_minutes': total_minutes,
            'total_hours': total_hours,
            'total_days_8h': total_days_8h,
            'total_days_4h': total_days_4h,
            'total_days_2h': total_days_2h,
            'total_pages': df['Páginas'].sum(),
            'total_words': df['Palabras'].sum(),
            'reading_speed': self.words_per_minute
        }

In [ ]:
# Crear la calculadora con la bibliografía específica
calculator = ReadingTimeCalculator(words_per_minute=180)

# Definir todos los documentos
documents = [
    Document("Marco General - DC Nivel Primario", 150, 300, 1.2),
    Document("Diseño Curricular 1° Ciclo (pág. 258-279)", 22, 350, 1.3),
    Document("Diseño Curricular 2° Ciclo (pág. 438-470)", 33, 350, 1.3),
    Document("Orientación Enseñanza 1° Ciclo (pág. 84-88)", 5, 400, 1.1),
    Document("Orientación Enseñanza 2° Ciclo (pág. 104-108)", 5, 400, 1.1),
    Document("Régimen Académico - Versión Final", 50, 300, 1.4),
    Document("Diseño Curricular NES (secciones)", 80, 320, 1.2),
    Document("Educación Tecnológica Secundaria", 60, 280, 1.1),
    Document("Tecnologías de la Información NES", 40, 300, 1.1),
    Document("Estatuto Docente (Cap. III, Art. 6-7)", 8, 250, 1.5),
    Document("Reglamento Escolar (arts. específicos)", 25, 280, 1.4)
]

# Agregar documentos a la calculadora
for doc in documents:
    calculator.add_document(doc)

print("✅ Calculadora configurada con todos los documentos de la bibliografía")

In [ ]:
# Obtener resultados detallados
df_results = calculator.get_results_dataframe()
total_results = calculator.calculate_total_time()

print("📊 RESULTADOS DETALLADOS POR DOCUMENTO:")
print("=" * 80)
display(df_results.round(2))

print("\n📈 RESUMEN TOTAL:")
print("=" * 50)
print(f"📚 Total documentos: {len(documents)}")
print(f"📄 Total páginas: {total_results['total_pages']:,}")
print(f"📝 Total palabras: {total_results['total_words']:,}")
print(f"⏱️  Tiempo total: {total_results['total_hours']:.1f} horas")
print(f"📅 Días (8h/día): {total_results['total_days_8h']:.1f} días")
print(f"📅 Días (4h/día): {total_results['total_days_4h']:.1f} días")
print(f"📅 Días (2h/día): {total_results['total_days_2h']:.1f} días")

In [ ]:
# Visualizar tiempo por documento
plt.figure(figsize=(12, 8))

# Gráfico de barras con tiempo por documento
plt.subplot(2, 2, 1)
plt.barh(df_results['Documento'], df_results['Horas'])
plt.xlabel('Tiempo (horas)')
plt.title('Tiempo de Lectura por Documento')
plt.xticks(rotation=45)
plt.tight_layout()

# Gráfico de torta con distribución del tiempo
plt.subplot(2, 2, 2)
plt.pie(df_results['Horas'], labels=df_results['Documento'], autopct='%1.1f%%')
plt.title('Distribución del Tiempo de Lectura')

# Gráfico de páginas por documento
plt.subplot(2, 2, 3)
plt.bar(range(len(df_results)), df_results['Páginas'])
plt.xlabel('Documentos')
plt.ylabel('Páginas')
plt.title('Páginas por Documento')
plt.xticks(range(len(df_results)), [f'Doc {i+1}' for i in range(len(df_results))])

# Gráfico de dificultad vs tiempo
plt.subplot(2, 2, 4)
plt.scatter(df_results['Dificultad'], df_results['Horas'], s=df_results['Páginas']*2)
plt.xlabel('Factor de Dificultad')
plt.ylabel('Tiempo (horas)')
plt.title('Dificultad vs Tiempo (tamaño = páginas)')

plt.tight_layout()
plt.show()

In [ ]:
# Análisis de sensibilidad: diferentes velocidades de lectura
speeds = [120, 150, 180, 200, 250, 300]
speed_results = []

for speed in speeds:
    calc_temp = ReadingTimeCalculator(words_per_minute=speed)
    for doc in documents:
        calc_temp.add_document(doc)
    
    results = calc_temp.calculate_total_time()
    speed_results.append({
        'Velocidad (ppm)': speed,
        'Horas Totales': results['total_hours'],
        'Días (8h/día)': results['total_days_8h'],
        'Días (4h/día)': results['total_days_4h'],
        'Días (2h/día)': results['total_days_2h']
    })

df_speeds = pd.DataFrame(speed_results)

print("🏃 ANÁLISIS DE VELOCIDADES DE LECTURA:")
print("=" * 60)
display(df_speeds.round(2))

# Gráfico de velocidades
plt.figure(figsize=(10, 6))
plt.plot(df_speeds['Velocidad (ppm)'], df_speeds['Horas Totales'], 'o-', linewidth=2, markersize=8)
plt.xlabel('Velocidad de Lectura (palabras por minuto)')
plt.ylabel('Tiempo Total (horas)')
plt.title('Tiempo Total vs Velocidad de Lectura')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Planificador de estudio
def create_study_plan(hours_per_day: float = 4.0, include_breaks: bool = True):
    """Crea un plan de estudio detallado"""
    
    # Factor de tiempo adicional para descansos y notas
    break_factor = 1.3 if include_breaks else 1.0
    
    df_plan = df_results.copy()
    df_plan['Horas Ajustadas'] = df_plan['Horas'] * break_factor
    df_plan['Días Necesarios'] = df_plan['Horas Ajustadas'] / hours_per_day
    
    # Ordenar por tiempo requerido (opcional: puedes cambiar el orden)
    df_plan = df_plan.sort_values('Horas Ajustadas', ascending=False)
    
    total_days = df_plan['Días Necesarios'].sum()
    
    print(f"📋 PLAN DE ESTUDIO - {hours_per_day}h/día")
    print(f"{'Con' if include_breaks else 'Sin'} tiempo extra para descansos y notas")
    print("=" * 70)
    
    cumulative_days = 0
    for idx, row in df_plan.iterrows():
        cumulative_days += row['Días Necesarios']
        print(f"📚 {row['Documento']}")
        print(f"   ⏱️  Tiempo: {row['Horas Ajustadas']:.1f}h ({row['Días Necesarios']:.1f} días)")
        print(f"   📅 Día acumulado: {cumulative_days:.1f}")
        print()
    
    print(f"🎯 TOTAL: {total_days:.1f} días de estudio")
    print(f"📆 Tiempo recomendado: {total_days + 2:.1f} días (incluye 2 días de repaso)")
    
    return df_plan

# Crear plan de estudio
study_plan = create_study_plan(hours_per_day=4.0, include_breaks=True)

In [ ]:
# Calculadora interactiva - puedes cambiar estos valores
print("🎛️ CALCULADORA INTERACTIVA")
print("=" * 50)
print("Cambia estos valores para personalizar el cálculo:")
print()

# Parámetros personalizables
mi_velocidad = 180  # Cambia este valor según tu velocidad de lectura
horas_por_dia = 4   # Cambia según tu tiempo disponible
incluir_descansos = True  # True para incluir tiempo extra

print(f"📖 Tu velocidad de lectura: {mi_velocidad} palabras/minuto")
print(f"⏰ Horas de estudio por día: {horas_por_dia}")
print(f"☕ Incluir tiempo para descansos: {'Sí' if incluir_descansos else 'No'}")
print()

# Recalcular con tus parámetros
mi_calculadora = ReadingTimeCalculator(words_per_minute=mi_velocidad)
for doc in documents:
    mi_calculadora.add_document(doc)

mis_resultados = mi_calculadora.calculate_total_time()
factor_descanso = 1.3 if incluir_descansos else 1.0
tiempo_ajustado = mis_resultados['total_hours'] * factor_descanso
dias_necesarios = tiempo_ajustado / horas_por_dia

print(f"🎯 TUS RESULTADOS PERSONALIZADOS:")
print(f"⏱️  Tiempo total de lectura: {mis_resultados['total_hours']:.1f} horas")
print(f"⏱️  Tiempo con descansos: {tiempo_ajustado:.1f} horas")
print(f"📅 Días necesarios: {dias_necesarios:.1f} días")
print(f"📅 Recomendación: {dias_necesarios + 2:.1f} días (incluye repaso)")
print()
print("💡 CONSEJOS:")
print("• Si necesitas más tiempo, reduce tu velocidad de lectura")
print("• Para textos muy técnicos, considera 120-150 ppm")
print("• Toma descansos de 10-15 min cada hora")
print("• Haz resúmenes mientras lees para mejor comprensión")

## 🎯 Conclusiones y Recomendaciones

### Tiempo Total Estimado:
- **Aproximadamente 16-20 horas de lectura pura**
- **2-3 días de estudio intensivo (8 horas/día)**
- **4-5 días de estudio normal (4 horas/día)**
- **8-10 días de estudio ligero (2 horas/día)**

### Estrategias Recomendadas:

1. **Priorización**: Comienza por los documentos más extensos (Marco General, Diseño Curricular NES)
2. **Técnica Pomodoro**: 45 min de lectura + 15 min descanso
3. **Toma de notas**: Dedica tiempo extra para resumir conceptos clave
4. **Repaso**: Planifica 1-2 días adicionales para revisar tus notas

### Ajustes Personalizados:
- **Velocidad lenta (120-150 ppm)**: Para textos muy técnicos
- **Velocidad normal (180-200 ppm)**: Para documentos académicos
- **Velocidad rápida (250+ ppm)**: Para repasos o textos familiares

### 📅 Plan de Estudio Sugerido:
1. **Día 1-2**: Marco General + Diseño Curricular NES
2. **Día 3**: Diseño Curricular 1° y 2° Ciclo
3. **Día 4**: Régimen Académico + Documentos Tecnológicos
4. **Día 5**: Estatuto Docente + Reglamento Escolar + Orientaciones
5. **Día 6-7**: Repaso y síntesis

¡Buena suerte con tu examen! 🍀